In [2]:
%matplotlib inline
from pylab import *
import pandas as pd
from sklearn import manifold
from sklearn import *

import numpy as np 
from numpy import *

C:\Users\User\Anaconda3\lib\site-packages\sklearn\lda.py:4: DeprecationWarning: lda.LDA has been moved to discriminant_analysis.LinearDiscriminantAnalysis in 0.17 and will be removed in 0.19
  "in 0.17 and will be removed in 0.19", DeprecationWarning)
C:\Users\User\Anaconda3\lib\site-packages\sklearn\qda.py:4: DeprecationWarning: qda.QDA has been moved to discriminant_analysis.QuadraticDiscriminantAnalysis in 0.17 and will be removed in 0.19.
  "in 0.17 and will be removed in 0.19.", DeprecationWarning)


 <b>Avec le prétraitement en R  des variables qualitatives et des valeurs manquantes : <b>


In [13]:


x_train= pd.read_csv("traincomplet.csv",sep=",")
y_train=pd.read_csv("sortie_train.csv",sep=",")
x_test=pd.read_csv("test.csv",sep=",")

In [14]:
#affichage

x_train.head()

,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,...,Q63,Q64,Q65,Q66,Q67,Q68,Q69,Q70,Q71,Q72
0,2,3,2,0,0,0,1,1,8,2286,...,0,0,0,1,0,1,0,1.0,0,0
1,2,3,1,0,0,0,2,1,166,2220,...,0,0,0,0,0,1,0,1.0,0,0
2,4,2,1,0,0,0,2,1,17,2265,...,0,0,0,0,0,1,0,1.0,0,0
3,2,3,2,0,0,0,2,1,16,2250,...,0,0,1,0,0,1,0,1.0,0,0
4,2,2,2,0,0,0,1,1,23,2305,...,0,0,0,1,0,0,1,1.0,0,0


In [15]:
y_train.head()

,ID,TARGET
0,0,0
1,1,0
2,2,0
3,3,1
4,4,0


In [16]:
x_test.head()

,C2,C3,C4,C5,C6,C7,C8,C9,C10,C11,...,Q63,Q64,Q65,Q66,Q67,Q68,Q69,Q70,Q71,Q72
0,2,2,1,0,0,0,1,1,128,2295,...,0,0,0,0,1,1,0,1,0,0
1,2,2,2,0,1,0,1,1,20,2275,...,0,0,0,1,0,1,0,0,1,0
2,2,2,2,0,0,0,1,1,4,2320,...,0,0,1,0,0,0,1,0,0,0
3,2,2,2,1,0,0,1,1,6,2277,...,0,0,0,0,0,1,1,1,0,0
4,2,4,2,0,0,0,2,1,9,2245,...,0,0,0,0,1,1,0,1,0,0


In [17]:
shape(x_test)

(21260, 85)

In [18]:

y_train=y_train.values[:,1]
shape(y_train)
#y_train
#selection = Selec.fit_transform(x_train,y_train)






(85529,)

In [19]:
x_train=x_train.values

<b>stratification de l'echantillon: 
En effet un test nous a montré qu'on avait autant de 1 que 0 à predire alors que l'échantillon d'apprentissage n'a pas du tout cette Stratification d'ou par une méthode de tirage aléatoire nous allons constituer une échantillon stratifié<b>


In [42]:


#identification des lignes avec des 0 et des 1
ind0=[0]*85530
for i in range(85529):
    ind0[i]=i*(y_train[i]==0)

ind0
v1=[0]
v2=[3]
for i in range(85529):
    if ind0[i]>0:
        v1.append(ind0[i])
    elif (ind0[i]==0)&(i>3):
        v2.append(i)
        
ind0=v1
ind1=v2
len(ind1)

from random import uniform
v=[0]*len(ind1)
for i in range(len(ind1)):
    u=uniform(0,1)
    v[i]=(u<0.5)*ind0[i]+(u>=0.5)*ind1[i]

tir=v
len(tir)


16558

In [9]:
#prediction par plus proche voisins sur l'echantillon stratifié avec la totalité de l'échantillon :

from sklearn import neighbors
mod = neighbors.KNeighborsClassifier(n_neighbors = 10, weights="uniform")
mod.fit(x_train[tir[1:10000]], y_train[tir[1:10000]])

#mini-cross validation avec un bloc plus petit (testset validation) :
#print("Taux de bien classés sur le 2ème bloc :") 
print(mod.score(x_train[tir[10001:12000]], y_train[tir[10001:12000]]))

0.544772386193


In [20]:

x_test=x_test.values

In [11]:
pred = mod.predict(x_test)
pred
sum(pred)#pour vérifier que la proportion de 1 prédit est bien en accord 


7596

<b>même avec une stratification les k plus proches voisins ont des resultats relativements mauvais <b>

<b>Prédiction par Support vecteur machine :<b>

In [12]:
#test du svm posons le modèle:


from sklearn import svm
mod_svm = svm.SVC(kernel='rbf')

mod_svm.fit(x_train[tir[1:12000],:], y_train[tir[1:12000]])
pred_svm = mod_svm.predict(x_train[tir[1:12000],:])


#Cross validation sur un bloc plus petit(testset validation)  :


print("Taux de bien classés sur les données de validation:") 
print(mod_svm.score(x_train[tir[12000:14000],:], y_train[tir[12000:14000]]))


Taux de bien classés sur les données de validation:
0.543


<b>Prédiction par Support vecteur machine :<b>

In [69]:
pred_test1=mod_svm.predict(x_test)
sum(pred_test1)


16156

 <b>création du modèle de Random forest  avec les variables sélectionnées et l'echantillon stratifié   <b>

In [67]:

from sklearn.ensemble import RandomForestClassifier
mod3= RandomForestClassifier(n_estimators=200,max_features=84,max_depth =11,criterion="gini")

mod3.fit(x_train, y_train,sample_weight=np.array([1 if i==0 else 10 for i in y_train ))
#mod3_RF = mod3.predict(x_train[tir[0:8500],:])

#mini-cross validationn sur un bloc plus petit(testset validation)  :

print("Taux de bien classés sur les données de validation :") 

print(mod3.score(x_train[tir[12001:15000],:], y_train[tir[12001:15000]],sample_weight=
                 np.array([1 if i==0 else 1 for i in y_train[tir[12001:15000]]])))


Taux de bien classés sur les données de validation :
0.651217072357


In [69]:
yn=pd.read_csv("0.659.csv",sep=",")
pred_vie=mod3.predict(x_test)
sum(pred_vie)

tab=pd.DataFrame(pred_vie,columns=["TARGETS"])
tab.to_csv('pyt.csv', header=True)
sum(pred_vie)

9727

In [41]:
# cross-validation sur le nombre d'arbre


modtest= RandomForestClassifier()

for i in [1,5,10,30,50,75,100,150,200,300]:
    modtest.set_params(n_estimators=i,max_features=80,max_depth =16,criterion="entropy")
    modtest.fit(x_train[tir[0:9000],:], y_train[tir[0:9000]])
    print (i, modtest.score(x_train[tir[9001:12000],:], y_train[tir[9001:12000]]))
    
    
#150 reste optimal pour le nombre d'arbre optimal 

1 0.560853617873
5 0.594198066022
10 0.616872290764
30 0.638212737579
50 0.625208402801
75 0.634544848283
100 0.627542514171
150 0.634544848283
200 0.631543847949
300 0.636545515172


<b>un test à la main nous a montré qu'utiliser la totalité des variables est ce qui stabilise au mieux la méthode<b>

In [38]:
# cross validation sur la profondeur  des arbres


modtest= RandomForestClassifier()

for i in [1,5,10,30,50,75,100,125,150,200]:
    modtest.set_params(n_estimators=150,max_features=84,max_depth = i,criterion="entropy")
    modtest.fit(x_train[tir[0:9000],:], y_train[tir[0:9000]])
    #validation
    print (i, modtest.score(x_train[tir[9001:15000],:], y_train[tir[9001:15000]]))
    
#10 reste la meilleure profondeur d'arbre  

1 0.574929154859
5 0.631938656443
10 0.64510751792
30 0.641940323387
50 0.641606934489
75 0.64327387898
100 0.637439573262
125 0.639773295549
150 0.643440573429
200 0.637606267711


<b>Random forest avec les paramètres  obtenus par cross-validation(test-set validation)<b>

In [51]:
#cross-valider param


from sklearn.ensemble import RandomForestClassifier
mod3= RandomForestClassifier(n_estimators=150,max_features=10,max_depth =84,criterion="entropy",class_weigt=[0.5,0.5])
mod3.fit(x_train[tir[0:9000],:], y_train[tir[0:9000]])

#évaluation sur un bloc plus petit (testset validation ):

print("Taux de bien classés sur les données de validation :") 

print(mod3.score(x_train[tir[9001:15000],:], y_train[tir[9001:15000]]))


Taux de bien classés sur les données de validation :
0.645274212369


 <b>création du fichier de soumission <b>: 

In [72]:


pred_test=mod3.predict(x_test)
sum(pred_test)
yn=pd.read_csv("0.659.csv",sep=",")
tab=pd.DataFrame(pred_test,columns=["TARGETS"])
tab.to_csv('pyt.csv', header=True)
sum(pred_test)


9390

 <b>sélection de variables sur la random forest (les variables les plus importantes ) <b>:

In [73]:



imp=mod3.feature_importances_
imp
p=[]
type(p)
for i in range(50):
    if imp[i]>0.01:
         p.append(i)
p
len(p)

29

 <b>   entrainement sur l'échantillon constitué de variables les plus importantes dans la random_forest:  <b>

In [74]:
#entrainnement sur l'échantillon constitué de variables les plus importantes dans la random_forest:

from sklearn.ensemble import RandomForestClassifier
x_train_new=x_train[:,p]
mod4= RandomForestClassifier(n_estimators=200,max_features=len(p),max_depth =101,criterion="entropy")
mod4.fit(x_train_new[tir[0:9000],:], y_train[tir[0:9000]])
print("Taux de bien classés sur les données de validation :") 

print(mod4.score(x_train_new[tir[9001:11622],], y_train[tir[9001:11622]]))

Taux de bien classés sur les données de validation :
0.637542922549


<b>creation du fichier de soumission <b>: 

In [19]:
pred_test2=mod4.predict(x_test[:,p])
sum(pred_test2)
#yn=pd.read_csv("0.659.csv",sep=",")
tab=pd.DataFrame(pred_test2,columns=["TARGETS"])
tab.to_csv('pyt.csv', header=True)
sum(pred_test2)


9605

<b>  mise en oeuvre du Gradient Boosting pour booster la performance des arbres succéssifs  <b> :

In [83]:

from sklearn.ensemble import GradientBoostingClassifier
mod5= GradientBoostingClassifier(n_estimators=150,max_features=84,max_depth =15,verbose=True)
mod5.fit(x_train[tir[0:9000],:], y_train[tir[0:9000]])
#mod5_appr = mod5.predict(x_train[tir[0:8000],:])
print("Taux de bien classés sur les données d'apprentissage :") 
#calcul du score sur deux echantillons de test (cross validation de la methode) avec 
#la même strat

#Cross-validation des paramètres (non automatisée car de multiple test de paramètres non fortuits ont étés éffectués )
print(mod5.score(x_train[tir[9001:12000],:], y_train[tir[9001:12000]]))



      Iter       Train Loss   Remaining Time 
         1           1.2627            2.63m
         2           1.1571            2.75m
         3           1.0591            2.91m
         4           0.9752            3.01m
         5           0.9041            3.08m
         6           0.8348            3.19m
         7           0.7802            3.22m
         8           0.7241            3.31m
         9           0.6702            3.42m
        10           0.6213            3.50m
        20           0.3060            3.64m
        30           0.1592            3.66m
        40           0.0810            3.59m
        50           0.0453            3.32m
        60           0.0251            3.04m
        70           0.0146            2.68m
        80           0.0078            2.36m
        90           0.0044            2.01m
       100           0.0026            1.64m
Taux de bien classés sur les données d'apprentissage :
0.637545848616


In [21]:
#validation sur un deuxième echantillon de même taille que le premier:


print(mod5.score(x_train[tir[12000:14000],:], y_train[tir[12000:14000]]))

0.6325


<b>creation du fichier de soumission <b>: 

In [22]:



#pred_test3=mod5.predict(x_test)
#sum(pred_test3)
#tab=pd.DataFrame(pred_test3,columns=["TARGETS"])
#tab.to_csv('pyt.csv', header=True)
#sum(pred_test3)





 <b>Blending: croisement des réponses des meilleures méthodes obtenues  <b>

 <b>on ne va garder que les 1 ayant eu un grand nombre de vote par les deux méthodes les plus plus puissantes que 
    nous ayons eu à tester et ceux avec leurs meilleurs paramètres  <b>

In [140]:

    
prm31=(mod3.set_params(max_depth=11,n_estimators=200)).predict(x_test)
prm32=(mod3.set_params(max_depth=15,n_estimators=175)).predict(x_test)
prm33=(mod3.set_params(max_depth=30,n_estimators=150)).predict(x_test)
prm34=(mod3.set_params(max_depth=50,n_estimators=150)).predict(x_test)
prm51=(mod5.set_params(max_depth=11,n_estimators=200)).predict(x_test)
prm52=(mod5.set_params(max_depth=15,n_estimators=175)).predict(x_test)
prm53=(mod5.set_params(max_depth=30,n_estimators=150)).predict(x_test)
prm54=(mod5.set_params(max_depth=50,n_estimators=150)).predict(x_test)
pred_meill=((pd.read_csv("0.659.csv",sep=",")).values)[:,1]
v=[0]*21260
v=prm31+prm32+prm33+prm34+prm51+prm52+prm53+prm54+pred_meill+pred

for i in range(len(prm31)):
    if v[i]>7:
        v[i]=1
    else :
        v[i]=pred_meill[i]        
sum(v)





c=[0]*21260
for i in range(len(prm31)):
    c[i]=v[i]+pred_meill[i]
    if c[i]>1:
            c[i]=1
    elif c[i]==0:
            c[i]=0
    elif c[i]==1 & pred_meill[i]==1 :
        c[i]=1
    else:
        c[i]=0
        

sum(c)





8815